<img src="images/nvidia_header.png" style="margin-left: -30px; width: 300px; float: left;">

# Accelerating End-to-End Data Science Workflows # 

## 02 - K-Means ##

**Table of Contents**
<br>
This notebook uses GPU-accelerated K-means to find the best locations for a fixed number of humanitarian supply airdrop depots. This notebook covers the below sections: 
1. [Environment](#Environment)
2. [Load Data](#Load-Data)
3. [K-Means Clustering](#K-Means-Clustering)
    * [Exercise #1 - Make Another `KMeans` Instance](#Exercise-#1---Make-Another-KMeans-Instance)
4. [Visualize the Clusters](#Visualize-the-Clusters)

## Environment ##
For the first time we import `cuml`, the RAPIDS GPU-accelerated library containing many common machine learning algorithms. We will be visualizing the results of your work in this notebook, so we also import `cuxfilter`.

In [ ]:
# DO NOT CHANGE THIS CELL
import cudf
import cuml

import cuxfilter as cxf

## Load Data ##
For this notebook we load again the cleaned UK population data--in this case, we are not specifically looking at counties, so we omit that column and just keep the grid coordinate columns.

In [ ]:
# DO NOT CHANGE THIS CELL
gdf = cudf.read_csv('./data/clean_uk_pop.csv', usecols=['easting', 'northing'])
print(gdf.dtypes)
gdf.shape

In [ ]:
gdf.head()

<a name='#s2-3'></a>
## K-Means Clustering ##
The unsupervised K-means clustering algorithm will look for a fixed number *k* of centroids in the data and clusters each point with its closest centroid. K-means can be effective when the number of clusters *k* is known or has a good estimate (such as from a model of the underlying mechanics of a problem).

Assume that in addition to knowing the distribution of the population, which we do, we would like to estimate the best locations to build a fixed number of humanitarian supply depots from which we can perform airdrops and reach the population most efficiently. We can use K-means, setting *k* to the number of supply depots available and fitting on the locations of the population, to identify candidate locations.

GPU-accelerated K-means is just as easy as its CPU-only scikit-learn counterpart. In this series of exercises, you will use it to optimize the locations for 5 supply depots.

`cuml.KMeans()` will initialize a K-means instance. Use it now to initialize a K-means instance called `km`, passing the named argument `n_clusters` set equal to our desired number `5`. Use the `km.fit` method to fit `km` to the population's locations by passing it the population data. After fitting, add the cluster labels back to the `gdf` in a new column named `cluster`. Finally, you can use `km.cluster_centers_` to see where the algorithm created the 5 centroids.

Below we train a K-means clustering algorithm to find 5 clusters. 

In [ ]:
# DO NOT CHANGE THIS CELL
# instantaite
km = cuml.KMeans(n_clusters=5)

# fit
km.fit(gdf)

# assign cluster as new column
gdf['cluster'] = km.labels_
km.cluster_centers_

<a name='#s2-e1'></a>
## Exercise #1 - Make Another `KMeans` Instance ##

**Instructions**: <br>
* Modify the `<FIXME>` only and execute the below cell to instantiate a K-means instance with 6 clusters.
* Modify the `<FIXME>` only and execute the cell below to fit the data. 

In [ ]:
km = cuml.KMeans(n_clusters=<<<<FIXME>>>>)

In [ ]:
km.<<<<FIXME>>>>(gdf)
gdf['cluster'] = km.labels_
km.cluster_centers_

Click ... for solution. 

<a id='#s2-4'></a>
## Visualize the Clusters ##
To help us understand where clusters are located, we make a visualization that separates them, using the same three steps as before.

Below we plot the clusters with cuxfilter. 

In [ ]:
# DO NOT CHANGE THIS CELL
# associate a data source with cuXfilter
cxf_data = cxf.DataFrame.from_dataframe(gdf)

# define charts
scatter_chart = cxf.charts.datashader.scatter(x='easting', y='northing')

# define widget using the `cluster` column for multiselect
# use the same technique to scale the scatterplot, then add a widget to let us select which cluster to look at
cluster_widget = cxf.charts.panel_widgets.multi_select('cluster')

In [ ]:
# create dashboard
dash = cxf_data.dashboard(charts=[scatter_chart],sidebar=[cluster_widget], theme=cxf.themes.dark, data_size_widget=True)

In [ ]:
dash.app()

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

**Well Done!** Let's move to the [next notebook](3-03_dbscan.ipynb). 

<img src="images/nvidia_header.png" style="margin-left: -30px; width: 300px; float: left;">